# Data Pre Processing

This is a notebook for intaking a raw data file, preprocessing it, and exporting it in a format in which it can easily be labeled.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/directoryident.tsv", sep="\t")
df.head()

,dn,lastname,givenname,middle,dob,address
0,"uid=madanafl,ou=People,dc=umich,dc=edu",Madanay,Farrah,L,1990091200Z,NaN
1,"uid=jocbran,ou=People,dc=umich,dc=edu",Branham,Jocelyn,NaN,20050616000000Z,{addr1=345A Infantry Ter}:{city=San Francisco}...
2,"uid=stoicabr,ou=People,dc=umich,dc=edu",Stoica,Brooke,NaN,20031007000000Z,{addr1=9051 Salem Farms Drive}:{city=South Lyo...
3,"uid=heavens,ou=People,dc=umich,dc=edu",Smith,Heaven,NaN,19880104000000Z,{addr1=6655 Jackson Rd Unit 191}:{city=Ann Arb...
4,"uid=funnim,ou=People,dc=umich,dc=edu",Funni,Matthew,NaN,1989121900Z,{addr1=2095 pauls way}:{city=Commerce twp}:{st...


## Preprocessing

### Cleanup

In [3]:
# Add unique ID column
# Uses the row number of original csv file as unique ID
df['Unique ID'] = df.index + 1

In [4]:
# Turn all NaN values into empty strings
df['address'] = df['address'].fillna(" ")
df['middle'] = df['middle'].fillna(" ")

In [5]:
df.head()

,dn,lastname,givenname,middle,dob,address,Unique ID
0,"uid=madanafl,ou=People,dc=umich,dc=edu",Madanay,Farrah,L,1990091200Z,,1
1,"uid=jocbran,ou=People,dc=umich,dc=edu",Branham,Jocelyn,,20050616000000Z,{addr1=345A Infantry Ter}:{city=San Francisco}...,2
2,"uid=stoicabr,ou=People,dc=umich,dc=edu",Stoica,Brooke,,20031007000000Z,{addr1=9051 Salem Farms Drive}:{city=South Lyo...,3
3,"uid=heavens,ou=People,dc=umich,dc=edu",Smith,Heaven,,19880104000000Z,{addr1=6655 Jackson Rd Unit 191}:{city=Ann Arb...,4
4,"uid=funnim,ou=People,dc=umich,dc=edu",Funni,Matthew,,1989121900Z,{addr1=2095 pauls way}:{city=Commerce twp}:{st...,5


In [6]:
# Convert all columns to string
df = df.astype(str)

In [7]:
# Function to parse a singular dn field
# Returns the uid in the dn
def parse_dn(dn):
    dn = dn.split(',')
    dn = [x.split('=') for x in dn]
    dn = {x[0].strip(): x[1].strip() for x in dn}
    return dn["uid"]

In [8]:
# Function to concatenate every value in the address 
# We want to concatenate all values inside curly braces
def concat_address(address):
    if address != ' ':
        address = address.split('}:{')
        try: 
            address = [x.split('=')[1] for x in address]
        except:
            return "DELETE"
        address = ' '.join(address)
    return address

In [9]:
# Use vectorization to set new values for dn and address columns
df['dn'] = df['dn'].apply(parse_dn)
df['address'] = df['address'].apply(concat_address)
df['dob'] = df['dob'].apply(lambda x : x[:8])

In [10]:
if "DELETE" in df['address'].values:
    df = df[df['address'] != "DELETE"]

df.head()

,dn,lastname,givenname,middle,dob,address,Unique ID
0,madanafl,Madanay,Farrah,L,19900912,,1
1,jocbran,Branham,Jocelyn,,20050616,345A Infantry Ter San Francisco CA 94129-1280 ...,2
2,stoicabr,Stoica,Brooke,,20031007,9051 Salem Farms Drive South Lyon MI 48178 UNI...,3
3,heavens,Smith,Heaven,,19880104,6655 Jackson Rd Unit 191 Ann Arbor MI 48103-95...,4
4,funnim,Funni,Matthew,,19891219,2095 pauls way Commerce twp MI 48390 UNITED ST...,5


In [11]:
# Change column names
df.columns = ['ID', 'Last Name', 'First Name', 'Middle Name', 'Date of Birth', 'Address', 'Unique ID']
# Reorder columns
df = df[['Unique ID', 'ID', 'First Name', 'Middle Name', 'Last Name', 'Date of Birth', 'Address']]

In [12]:
df.head()

,Unique ID,ID,First Name,Middle Name,Last Name,Date of Birth,Address
0,1,madanafl,Farrah,L,Madanay,19900912,
1,2,jocbran,Jocelyn,,Branham,20050616,345A Infantry Ter San Francisco CA 94129-1280 ...
2,3,stoicabr,Brooke,,Stoica,20031007,9051 Salem Farms Drive South Lyon MI 48178 UNI...
3,4,heavens,Heaven,,Smith,19880104,6655 Jackson Rd Unit 191 Ann Arbor MI 48103-95...
4,5,funnim,Matthew,,Funni,19891219,2095 pauls way Commerce twp MI 48390 UNITED ST...


### Similarity Measures

In [13]:
# Sample 200 records from df into new df
df_sample = df.sample(100)

In [14]:
df_sample.head()

,Unique ID,ID,First Name,Middle Name,Last Name,Date of Birth,Address
657749,657750,swaltz,Susan,E,Waltz,19511009,202 E Riverside St Williamston MI 48895 USA}
88258,88259,nnick,Nick,,Nelson,19911029,
155929,155930,leomed,Roberto,Manuel Leonardo,Medrano,19971107,611 E University Ave Unit 595 Ann Arbor MI 481...
77734,77735,cornelwo,Cornelius,,Wood,19750622,2360 Ellsworth Road Apt. 201 Ypsilanti MI 4819...
316906,316907,chadaw,Chad,Austin,Williams,19901106,1507 Copper Mine Psge Fort Wayne IN 46845-0128...


In [15]:
# Remove sampled records from df
df = df.drop(df_sample.index)

With a sample df, we can now build a similarity measure df by comparing each record in the sample df with the other records.

In [16]:
import Levenshtein as lev

In [17]:
"""
Returns normalized levenshtein distance between two strings
"""
def levenshtein_distance(
        s1: str,
        s2: str
) -> int:
    distance = lev.distance(s1, s2)
    try:
        return 1 - distance / float(max(len(s1), len(s2)))
    except ZeroDivisionError:
        return 0

In [18]:
"""
Returns Levenshtein distance between each field of two rows
Assumes following format for rows:
    row = pd.Series([First Name, Middle Name, Last Name, DOB, Address, ID])
"""
def row_similarity(
        row_1: pd.Series, 
        df_2: pd.DataFrame
    ) -> pd.DataFrame:

    # Compute similarity measures for each column using Levenshtein distance
    first_name_similarity = df_2['First Name'].apply(lambda x : levenshtein_distance(row_1['First Name'], x))
    middle_name_similarity = df_2['Middle Name'].apply(lambda x : levenshtein_distance(row_1['Middle Name'], x))
    last_name_similarity = df_2['Last Name'].apply(lambda x : levenshtein_distance(row_1['Last Name'], x))
    dob_similarity = df_2['Date of Birth'].apply(lambda x : levenshtein_distance(row_1['Date of Birth'], x))
    address_similarity = df_2['Address'].apply(lambda x : levenshtein_distance(row_1['Address'], x))
    id_similarity = df_2['ID'].apply(lambda x : levenshtein_distance(row_1['ID'], x))

    # Return DataFrame with similarity measures
    return pd.DataFrame({
        'UNIQ_ID1': row_1['Unique ID'],
        'UNIQ_ID2': df_2['Unique ID'],
        'ID1': row_1['ID'],
        'ID2': df_2['ID'],
        'ID Similarity': id_similarity,
        'First Name 1' : row_1['First Name'],
        'First Name 2' : df_2['First Name'],
        'First Name Similarity': first_name_similarity,
        'Middle Name 1' : row_1['Middle Name'],
        'Middle Name 2' : df_2['Middle Name'],
        'Middle Name Similarity': middle_name_similarity,
        'Last Name 1' : row_1['Last Name'],
        'Last Name 2' : df_2['Last Name'],
        'Last Name Similarity': last_name_similarity,
        'Date of Birth 1' : row_1['Date of Birth'],
        'Date of Birth 2' : df_2['Date of Birth'],
        'Date of Birth Similarity': dob_similarity,
        'Address 1' : row_1['Address'],
        'Address 2' : df_2['Address'],
        'Address Similarity': address_similarity
    })


In [19]:
"""
Builds similarity measure between records in two df
Creates a new df from the two df with the following columns:
    - ID1: ID of record in df1
    - ID2: ID of record in df2
    - First Name Similarity: Normalized levenshtein distance between first names
    - Middle Name Similarity: Normalized levenshtein distance between middle names
    - Last Name Similarity: Normalized levenshtein distance between last names
    - Date of Birth Similarity: Normalized levenshtein distance between dates of birth
    - Address Similarity: Normalized levenshtein distance between addresses
    - ID Similarity: Normalized levenshtein distance between IDs
"""
def build_similarity_df (
        df_1: pd.DataFrame,
        df_2: pd.DataFrame
) -> pd.DataFrame:
    new_df = pd.DataFrame(columns=[ 'UNIQ_ID1',
                                    'UNIQ_ID2',
                                    'ID1', 
                                   'ID2', 
                                   'ID Similarity',
                                   'First Name 1',
                                   'First Name 2',
                                   'First Name Similarity', 
                                   'Middle Name 1',
                                    'Middle Name 2',
                                   'Middle Name Similarity',
                                    'Last Name 1',
                                    'Last Name 2', 
                                   'Last Name Similarity', 
                                    'Date of Birth 1',
                                    'Date of Birth 2',
                                   'Date of Birth Similarity', 
                                    'Address 1',
                                    'Address 2',
                                   'Address Similarity'])

    # Convert ID columns to string
    df_1["ID"] = df_1["ID"].astype(str)
    df_2["ID"] = df_2["ID"].astype(str)

    def apply_row_similarity(row, new_df):
        sim = row_similarity(row, df_2)
        new_df = pd.concat([new_df, sim], ignore_index=True)
        return new_df

    # Use vectorization to compute similarity between each row in df_1 and df_2
    new_df = df_1.apply(apply_row_similarity, args=(new_df,), axis=1).reset_index(drop=True)

    # New df is a series of dfs, so we need to concatenate them
    new_df = pd.concat(new_df.to_list(), ignore_index=True)

    return new_df

In [20]:
similarity_df = build_similarity_df(df_sample,  df)
similarity_df.head()

,UNIQ_ID1,UNIQ_ID2,ID1,ID2,ID Similarity,First Name 1,First Name 2,First Name Similarity,Middle Name 1,Middle Name 2,Middle Name Similarity,Last Name 1,Last Name 2,Last Name Similarity,Date of Birth 1,Date of Birth 2,Date of Birth Similarity,Address 1,Address 2,Address Similarity
0,657750,1,swaltz,madanafl,0.125000,Susan,Farrah,0.166667,E,L,0.0,Waltz,Madanay,0.142857,19511009,19900912,0.375,202 E Riverside St Williamston MI 48895 USA},,0.022727
1,657750,2,swaltz,jocbran,0.000000,Susan,Jocelyn,0.142857,E,,0.0,Waltz,Branham,0.142857,19511009,20050616,0.000,202 E Riverside St Williamston MI 48895 USA},345A Infantry Ter San Francisco CA 94129-1280 ...,0.234375
2,657750,3,swaltz,stoicabr,0.125000,Susan,Brooke,0.000000,E,,0.0,Waltz,Stoica,0.000000,19511009,20031007,0.375,202 E Riverside St Williamston MI 48895 USA},9051 Salem Farms Drive South Lyon MI 48178 UNI...,0.295082
3,657750,4,swaltz,heavens,0.142857,Susan,Heaven,0.166667,E,,0.0,Waltz,Smith,0.200000,19511009,19880104,0.375,202 E Riverside St Williamston MI 48895 USA},6655 Jackson Rd Unit 191 Ann Arbor MI 48103-95...,0.268657
4,657750,5,swaltz,funnim,0.000000,Susan,Matthew,0.000000,E,,0.0,Waltz,Funni,0.000000,19511009,19891219,0.500,202 E Riverside St Williamston MI 48895 USA},2095 pauls way Commerce twp MI 48390 UNITED ST...,0.254545


### Discretization

### WPKID

In [22]:
wpkid_df = similarity_df.copy()

In [23]:
# holds bin ranges for each similarity attribute
bin_ranges = {'First Name Similarity' : [],
        'Middle Name Similarity' : [],
        'Last Name Similarity' : [],
        'Date of Birth Similarity' : [],
        'Address Similarity' : [],
        'ID Similarity' : []}

In [24]:
#for solving systems of equations
import sympy as sp 
import math


t = # of intervals<br>
s = interval size<br>
n = # of instances in training dataset<br>

s * t = n<br>
s - 30 = t<br>


In [25]:
# WPKID function to solve for s and t given n, returns pair {s, t}
def interval_size_and_num(n) -> tuple:
    s, t = sp.symbols('s t')
    equation1 = sp.Eq(s * t, n)
    equation2 = sp.Eq(s - 30, t)  
    solution = sp.solve((equation1, equation2), (s, t))
    s_val = solution[0][1] * -1
    t_val = solution[1][1]

    s_int = math.floor(s_val)
    t_int = math.floor(t_val)
    return s_int, t_int

In [26]:
# function that takes in a column and discretizes it, saving bin ranges
def discretize_column(column, df):
    # separate zeros from nonzeros
    zeros_df = wpkid_df[wpkid_df[column] == 0].copy()
    nonzeros_df = wpkid_df[wpkid_df[column] != 0].copy()

    #get num_instances and num_intervals for nonzeros
    num_instances = interval_size_and_num(len(nonzeros_df))[0]
    num_intervals = interval_size_and_num(len(nonzeros_df))[1]

    # get bin ranges for nonzeros
    nonzeros_df[column], bin = pd.qcut(nonzeros_df[column], 
                                        q=num_intervals, 
                                        labels=False,
                                        retbins=True,
                                        duplicates='drop') 
    
    #add 1 to every value to switch to index 1
    nonzeros_df[column] = nonzeros_df[column] + 1

    #save bin ranges
    bin_ranges[column].extend(bin.tolist())

    #recombine zero and nonzero dataframes
    df.loc[df[column] == 0, column] = zeros_df[column]
    df.loc[df[column] != 0, column] = nonzeros_df[column] 

    return df

In [27]:
wpkid_df = discretize_column('First Name Similarity', wpkid_df)
wpkid_df = discretize_column('Middle Name Similarity', wpkid_df)
wpkid_df = discretize_column('Last Name Similarity', wpkid_df)
wpkid_df = discretize_column('Date of Birth Similarity', wpkid_df)
wpkid_df = discretize_column('Address Similarity', wpkid_df)
wpkid_df = discretize_column('ID Similarity', wpkid_df)

In [28]:
"""insert 0 at the beginning of each bin range"""
for key in bin_ranges:
    bin_ranges[key].insert(0, 0)

In [29]:
wpkid_df.head()

,UNIQ_ID1,UNIQ_ID2,ID1,ID2,ID Similarity,First Name 1,First Name 2,First Name Similarity,Middle Name 1,Middle Name 2,Middle Name Similarity,Last Name 1,Last Name 2,Last Name Similarity,Date of Birth 1,Date of Birth 2,Date of Birth Similarity,Address 1,Address 2,Address Similarity
0,657750,1,swaltz,madanafl,1.0,Susan,Farrah,21.0,E,L,0.0,Waltz,Madanay,20.0,19511009,19900912,2.0,202 E Riverside St Williamston MI 48895 USA},,62.0
1,657750,2,swaltz,jocbran,0.0,Susan,Jocelyn,17.0,E,,0.0,Waltz,Branham,20.0,19511009,20050616,0.0,202 E Riverside St Williamston MI 48895 USA},345A Infantry Ter San Francisco CA 94129-1280 ...,381.0
2,657750,3,swaltz,stoicabr,1.0,Susan,Brooke,0.0,E,,0.0,Waltz,Stoica,0.0,19511009,20031007,2.0,202 E Riverside St Williamston MI 48895 USA},9051 Salem Farms Drive South Lyon MI 48178 UNI...,465.0
3,657750,4,swaltz,heavens,1.0,Susan,Heaven,21.0,E,,0.0,Waltz,Smith,29.0,19511009,19880104,2.0,202 E Riverside St Williamston MI 48895 USA},6655 Jackson Rd Unit 191 Ann Arbor MI 48103-95...,432.0
4,657750,5,swaltz,funnim,0.0,Susan,Matthew,0.0,E,,0.0,Waltz,Funni,0.0,19511009,19891219,3.0,202 E Riverside St Williamston MI 48895 USA},2095 pauls way Commerce twp MI 48390 UNITED ST...,414.0


In [30]:
for key, value in bin_ranges.items():
    print(key, value)

First Name Similarity [0, 0.033333333333333326, 0.052631578947368474, 0.05882352941176472, 0.0625, 0.06666666666666665, 0.0714285714285714, 0.07692307692307687, 0.08333333333333337, 0.09090909090909094, 0.09523809523809523, 0.09999999999999998, 0.10526315789473684, 0.11111111111111116, 0.11764705882352944, 0.125, 0.1333333333333333, 0.13636363636363635, 0.1428571428571429, 0.15000000000000002, 0.15384615384615385, 0.16000000000000003, 0.16666666666666663, 0.17647058823529416, 0.18181818181818177, 0.1875, 0.19999999999999996, 0.2142857142857143, 0.2222222222222222, 0.23076923076923073, 0.23529411764705888, 0.25, 0.2666666666666667, 0.2727272727272727, 0.2857142857142857, 0.30000000000000004, 0.3125, 0.33333333333333337, 0.36363636363636365, 0.375, 0.4, 0.4285714285714286, 0.4444444444444444, 0.4545454545454546, 0.5, 0.5555555555555556, 0.5714285714285714, 0.6, 0.625, 0.6363636363636364, 0.6666666666666667, 0.7142857142857143, 0.75, 0.8, 0.8333333333333334, 0.8571428571428572, 0.875, 0.8

In [ ]:
# export wpkid_df to csv 
wpkid_df.to_csv('data/unlabeledident.csv', index=False)